## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
app_df_dropped = application_df.drop(columns = ["EIN", "NAME"])
app_df_dropped.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
app_df_dropped.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = app_df_dropped["APPLICATION_TYPE"].value_counts()
print(app_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value (count < 100) and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Saving the top 9 index values
application_types_to_replace = app_type_counts.index[9:].tolist()

In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    app_df_dropped['APPLICATION_TYPE'] = app_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
app_df_dropped['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_counts = app_df_dropped["CLASSIFICATION"].value_counts()
print(class_counts.head(20))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value (count < 500) and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Saving the top 6 values in this case
classifications_to_replace = class_counts.index[6:].tolist()

In [9]:
# Replace in dataframe
for cls in classifications_to_replace:
    app_df_dropped['CLASSIFICATION'] = app_df_dropped['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df_dropped['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(app_df_dropped, columns = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION",\
                                                                      "USE_CASE", "ORGANIZATION", "INCOME_AMT",\
                                                                     "SPECIAL_CONSIDERATIONS"])

In [11]:
# Split our preprocessed data into our features and target arrays
X = encoded_df.drop(columns = "IS_SUCCESSFUL").values
y = encoded_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
hl1_units = 16
hl2_units = 16

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hl1_units, activation = "relu", input_dim = 45))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hl2_units, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 16)                736       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 967us/step - loss: 0.5979 - accuracy: 0.7021
Epoch 2/100
804/804 [==============================] - 1s 943us/step - loss: 0.5584 - accuracy: 0.7265
Epoch 3/100
804/804 [==============================] - 1s 942us/step - loss: 0.5540 - accuracy: 0.7288
Epoch 4/100
804/804 [==============================] - 1s 940us/step - loss: 0.5519 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 1s 938us/step - loss: 0.5505 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 1s 938us/step - loss: 0.5493 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 1s 937us/step - loss: 0.5479 - accuracy: 0.7321
Epoch 8/100
804/804 [==============================] - 1s 947us/step - loss: 0.5478 - accuracy: 0.7308
Epoch 9/100
804/804 [==============================] - 1s 942us/step - loss: 0.5470 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 1s 937us/step - l

804/804 [==============================] - 1s 937us/step - loss: 0.5383 - accuracy: 0.7374
Epoch 80/100
804/804 [==============================] - 1s 939us/step - loss: 0.5384 - accuracy: 0.7385
Epoch 81/100
804/804 [==============================] - 1s 938us/step - loss: 0.5383 - accuracy: 0.7378
Epoch 82/100
804/804 [==============================] - 1s 937us/step - loss: 0.5380 - accuracy: 0.7364
Epoch 83/100
804/804 [==============================] - 1s 947us/step - loss: 0.5385 - accuracy: 0.7376
Epoch 84/100
804/804 [==============================] - 1s 942us/step - loss: 0.5380 - accuracy: 0.7377
Epoch 85/100
804/804 [==============================] - 1s 940us/step - loss: 0.5380 - accuracy: 0.7382
Epoch 86/100
804/804 [==============================] - 1s 943us/step - loss: 0.5381 - accuracy: 0.7377
Epoch 87/100
804/804 [==============================] - 1s 938us/step - loss: 0.5383 - accuracy: 0.7385
Epoch 88/100
804/804 [==============================] - 1s 939us/step - loss:

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7303 - 311ms/epoch - 1ms/step
Loss: 0.554117739200592, Accuracy: 0.7302623987197876


In [20]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")